In [2]:
import pandas as pd

# Load full-feature dataset
df = pd.read_excel("final_density_data.xlsx")

# Step 1: Reverse-calculate missing base columns
df["TOT_P"] = df["Population Density"] * df["Area"]
df["P_LIT"] = df["Literate Density"] * df["Area"]
df["P_06"] = df["Children Density"] * df["Area"]

# Step 2: Calculate ratios
df["Literate Ratio"] = df["P_LIT"] / df["TOT_P"]
df["Children Ratio"] = df["P_06"] / df["TOT_P"]

In [3]:
# Step 3: Assign levels using quantiles
def bucket_column(series):
    q1 = series.quantile(0.33)
    q2 = series.quantile(0.66)
    def assign_level(x):
        if x <= q1:
            return "Low"
        elif x <= q2:
            return "Medium"
        else:
            return "High"
    return series.apply(assign_level)

In [4]:
# Bucket each feature
df["Population_per_sq_km_level"] = bucket_column(df["Population Density"])
df["Estimated_Area_level"] = bucket_column(df["Area"])
df["Literate_Density_level"] = bucket_column(df["Literate Density"])
df["Children_Density_level"] = bucket_column(df["Children Density"])
#df["Literate_Ratio_level"] = bucket_column(df["Literate Ratio"])
#df["Children_Ratio_level"] = bucket_column(df["Children Ratio"])

Red = any row matching at least 3 out of 4 conditions.

Yellow = 2 conditions.

Green = 0 or 1.

In [5]:
def assign_zone(row):
    score = 0
    if row["Population_per_sq_km_level"] == "High":
        score += 1
    if row["Estimated_Area_level"] == "Low":
        score += 1
    if row["Literate_Density_level"] == "Low":
        score += 1
    if row["Children_Density_level"] == "High":
        score += 1

    if score >= 3:
        return "Red"
    elif score == 2:
        return "Yellow"
    else:
        return "Green"


In [6]:
# Apply the function
df["Zone"] = df.apply(assign_zone, axis=1)

# Optional: drop helper columns
df.drop(columns=[
    "TOT_P", "P_LIT", "P_06",
    "Literate Ratio", "Children Ratio",
    "Population_per_sq_km_level", "Estimated_Area_level",
    "Literate_Density_level", "Children_Density_level",
    "Literate_Ratio_level", "Children_Ratio_level"
], inplace=True)

# Save final version
df.to_excel("districts_with_zones_instructor_method-test-4.xlsx", index=False)


In [7]:
df = pd.read_excel("districts_with_zones_instructor_method-test-4.xlsx")

In [8]:
df.columns

Index(['District Code', 'District Code.1', 'State Code', 'District Name',
       'Area', 'Population Density', 'Literate Density', 'Children Density',
       'Zone'],
      dtype='object')

In [9]:
df.Zone.value_counts()

Green     392
Yellow    133
Red       115
Name: Zone, dtype: int64

In [10]:
df[df["Zone"] == "Red"]

,District Code,District Code.1,State Code,District Name,Area,Population Density,Literate Density,Children Density,Zone
8,9,9,1,Bandipore,345.000000,1136.904348,539.069565,178.997101,Red
9,10,10,1,Srinagar,1979.000000,624.976756,378.263770,79.989894,Red
10,11,11,1,Ganderbal,259.000000,1148.440154,553.189189,195.343629,Red
12,13,13,1,Shupiyan,312.000000,853.253205,437.500000,133.163462,Red
14,15,15,1,Kulgam,410.000000,1035.324390,509.963415,174.392683,Red
...,...,...,...,...,...,...,...,...,...
628,629,629,33,Kanniyakumari,1683.998875,1110.674138,919.678761,108.283920,Red
633,634,634,34,Yanam,30.000000,1854.200000,1309.133333,206.800000,Red
634,635,635,34,Puducherry,294.000000,3232.275510,2471.595238,339.585034,Red
635,636,636,34,Mahe,9.000000,4646.222222,4052.222222,505.888889,Red
